## Файнтюнинг rugpt3small модели от сбербанка



In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

c:\Users\m9lco\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#загрузка и инициализации модели и токенизатора. Модель поменьше, чтобы хватило памяти gpu
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path, bos_token='<s>', eos_token='</s>', pad_token='<pad>')
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [3]:
#Найденный датасет приводится к виду '<s> ' + anek + ' </s>\n', где [SJ] и [EJ] - теги начала и конца анекдота
data_path = 'anek.txt'
train_path = 'train_dataset.txt'
with open(data_path,'r', encoding='utf-8') as input, open(train_path, "w", encoding="utf-8") as output:
    data = input.read()
    data_split = data.split('\n\n')
    i=0
    for anek in data_split:
        i+=1
        anek = anek[15:]
        output.write('<s> ' + anek + ' </s>\n')
        if i == 800:
            break

In [4]:
#Создаю датасет и даталоадер
from transformers import TextDataset, DataCollatorForLanguageModeling

train_dataset = TextDataset(tokenizer=tokenizer,file_path=train_path,block_size=64)
  
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

c:\Users\m9lco\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\data\datasets\language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [5]:
#Создаю аргументы для обучение
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./GPTftd", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=50, #Количество полных проходов через обучающий набор данных
    per_device_train_batch_size=8, #Размер выборка, которую необходимо обработать перед обновлением внутренних параметров модели для обучения
    per_device_eval_batch_size=8,  #Размер выборка, которую необходимо обработать перед обновлением внутренних параметров модели для оценки
    warmup_steps=10,#Количество шагов роста скорости обучения с низкой скорости до постоянной скорости после этого.
    gradient_accumulation_steps=8, #При использовании накопления градиента расчет градиента выполняется небольшими шагами, а не сразу для пакета. Позволяет использовать выборки большего размера
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)


In [6]:
#Процесс обучения
trainer.train()

  1%|          | 500/54450 [04:31<7:33:17,  1.98it/s]

{'loss': 2.9312, 'learning_rate': 9.909992652461427e-06, 'epoch': 0.46}


  2%|▏         | 1000/54450 [08:51<8:02:34,  1.85it/s]

{'loss': 2.8521, 'learning_rate': 9.818148420279208e-06, 'epoch': 0.92}


  3%|▎         | 1500/54450 [13:25<7:30:54,  1.96it/s] 

...

 99%|█████████▉| 54000/54450 [8:08:52<03:37,  2.07it/s]

{'loss': 2.1661, 'learning_rate': 8.265980896399706e-08, 'epoch': 49.58}


100%|██████████| 54450/54450 [8:12:32<00:00,  1.84it/s]

{'train_runtime': 29552.3286, 'train_samples_per_second': 117.933, 'train_steps_per_second': 1.842, 'train_loss': 2.360816237884886, 'epoch': 49.99}


TrainOutput(global_step=54450, training_loss=2.360816237884886, metrics={'train_runtime': 29552.3286, 'train_samples_per_second': 117.933, 'train_steps_per_second': 1.842, 'train_loss': 2.360816237884886, 'epoch': 49.99})

In [97]:
text = "<s> Медведь сел в машину</s>"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=2,
                        temperature=1.5,
                        top_p=0.9,
                        max_length=70,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
anek_without_tr = ""
for i in range(0, len(generated_text) - 1):
    if ((generated_text[i] == "[") and (generated_text[i+1] == "E") and
        (generated_text[i+2] == "J") and (generated_text[i+3] == "]")):
        break 
    elif ((generated_text[i] == "[") or (generated_text[i] == "S") or
        (generated_text[i] == "J") or (generated_text[i] == "]") or 
        (generated_text[i] == "<") or (generated_text[i] == ">") or 
        (generated_text[i] == "s") or (generated_text[i] == "/")):
        i+=1
    elif generated_text[i] == "-" and generated_text[i+1] == " ":
        anek_without_tr += "\n-"
    else:
        anek_without_tr += generated_text[i]
print(anek_without_tr)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 Медведь сел в машину. 


In [67]:
output_dir = 'GTPpretrained'

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('GTPpretrained\\tokenizer_config.json',
 'GTPpretrained\\special_tokens_map.json',
 'GTPpretrained\\vocab.json',
 'GTPpretrained\\merges.txt',
 'GTPpretrained\\added_tokens.json')